## CONNECTING TO GOOGLE CLOUD STORAGE FROM LOCAL SPARK
In the last notebook we created a local session for spark and used the data saved locally. This time we want to pull data from GCS. In order to do that, we need to first upload the parquet files we created before to GCS. -m is for running multi-thread.

gsutil cp -m -r data/pq/ gs://ny-taxi-data-for-spark/pq

#### GET THE DATA FROM GOOGLE CLOUD
Download the Spark GCS connector so that spark will understand how to read this URL 
You need the .jar file - cloud storage connector for Hadoop3 

https://cloud.google.com/dataproc/docs/concepts/connectors/cloud-storage


In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf
from pyspark.context import SparkContext

import os

In [4]:
spark_home = os.environ.get('SPARK_HOME')
jar_file = os.path.join(spark_home, 'lib', 'gcs-connector-hadoop3-latest.jar')
print(spark_home)

/home/lulu/spark/spark-3.5.0-bin-hadoop3


In [15]:
spark.stop()

In [5]:
credentials_location = '/home/lulu/.gcp-keys/dbt-ny-taxi-key.json'

conf = SparkConf() \
    .setMaster('local[*]') \
    .setAppName('test') \
    .set("spark.jars", jar_file) \
    .set("spark.hadoop.google.cloud.auth.service.account.enable", "true") \
    .set("spark.hadoop.google.cloud.auth.service.account.json.keyfile", credentials_location)

## Create the Spark Context. 
In the previous examples, we initiated a Spark application with the SparkSession.builder() method, which creates a SparkContext. For connecting to GCS, it is common practice to first explicitly define the sparkContext with Hadoop config properties related to GCS and then create a session.  

The abstract (URIs gs://) and concrete FileSystem implementations are defined here with classes in the connector specified in the config. Using this implementation when interacting with GCS ensures that Spark Hadoop can read and write to GCS correctly. 

confirmed that the class was in the version of the jar file I downloaded 
jar tf gcs-connector-hadoop3-latest.jar | grep GoogleHadoopFS


In [6]:
sc = SparkContext(conf=conf)

hadoop_conf = sc._jsc.hadoopConfiguration()

hadoop_conf.set("fs.AbstractFileSystem.gs.impl",  "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")
hadoop_conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
hadoop_conf.set("fs.gs.auth.service.account.json.keyfile", credentials_location)
hadoop_conf.set("fs.gs.auth.service.account.enable", "true")

24/04/03 10:20:41 WARN Utils: Your hostname, pepper resolves to a loopback address: 127.0.1.1; using 192.168.178.64 instead (on interface wlp2s0)
24/04/03 10:20:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/04/03 10:20:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/04/03 10:20:42 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


We have a configuration and context. Now we need to create a session. 

In [7]:
spark = SparkSession.builder \
    .config(conf=sc.getConf()) \
    .getOrCreate()

In [8]:
#!gsutil ls gs://ny-taxi-data-for-spark/pq/green/
!gsutil ls gs://ny-taxi-data-for-spark/

gs://ny-taxi-data-for-spark/collisions.parquet
gs://ny-taxi-data-for-spark/code/
gs://ny-taxi-data-for-spark/pq/


In [9]:
#df_green = spark.read.parquet('gs://ny-taxi-data-for-spark/pq/green/*/*')
df_collisions = spark.read.parquet('gs://ny-taxi-data-for-spark/collisions.parquet')
#df_green = spark.read.parquet('gs://mage-zoomcamp-lulu-eu/green/*')

In [ ]:
df_green = spark.read.parquet('gs://ny-taxi-data-for-spark/pq/green/*/*')
df_green.show(5)

I had auth issues and had to add storage object permissions in GCP. 

In [10]:
#df_green.show(5)
df_collisions.show(5)

24/04/03 10:21:16 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+--------------------+----------+--------------------+---------------+-------------------------+------------------------+-----------------------------+----------------------------+-------------------------+------------------------+--------------------------+-------------------------+-----------------------------+-----------------------------+------------+------------------+------------------+--------+--------+---------+----------+--------------------+-----------------------------+-------------------+-----------------------------+-------------------+-----------------------------+-------------------+
|          crash_date|crash_time|      on_street_name|off_street_name|number_of_persons_injured|number_of_persons_killed|number_of_pedestrians_injured|number_of_pedestrians_killed|number_of_cyclist_injured|number_of_cyclist_killed|number_of_motorist_injured|number_of_motorist_killed|contributing_factor_vehicle_1|contributing_factor_vehicle_2|collision_id|vehicle_type_code1|vehicle_type_code2